In [1]:
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#ω
omega_l=[2,2,2.5,2.5,3,3,3.5,3.5,5,5,5,5,3.5,3.5,3,3,2,2,2,2] #angular velocity (rad/s) out put of left wheel encoder
omega_r=[2,2,2,2,2.5,2.5,3.5,3.5,5,5,5,5,4.5,4.5,3.5,3.5,2.5,2.5,2,2] #angular velocity (rad/s) out put of right wheel encoder

In [3]:
L=0.5 #meters
robot_width=2*L #meters
wheel_radius = 0.1 #meters
delta_time = 0.5 #seconds

In [4]:
init_pos=(0,0,0)

In [23]:
class differential_drive_robot():
    
    def __init__(self,robot_width,wheel_radius,init_pos,delta_time):
        '''
        the initialization of the mobile robot object
        '''
        
        self.v_l=[0]
        self.v_r=[0]
        self.del_s_l=[0]
        self.del_s_r=[0]
        self.delta_s=[0]
        self.time=[0]
        self.current_pos=init_pos
        self.path=[init_pos]
        self.robot_width=robot_width
        self.wheel_radius=wheel_radius
        self.init_pos=init_pos
        self.delta_time=delta_time
    def displacement(self, vel_l,vel_r):
        delta_s_l=vel_l*self.delta_time
        delta_s_r=vel_r*self.delta_time
        
        self.time.append(self.time[len(self.time)-1]+self.delta_time)
        
        self.del_s_l.append(delta_s_l)
        self.del_s_r.append(delta_s_r)
        
        self.delta_s.append((delta_s_l+delta_s_r)/2)
        
    def velocity(self,left_omega,right_omega):
        vel_l=left_omega*wheel_radius
        vel_r=right_omega*wheel_radius
        
        self.v_l.append(vel_l)
        self.v_r.append(vel_r)
        
        self.displacement(vel_l,vel_r)
            
        
        

In [26]:
if __name__ == '__main__':
    '''main'''
    mobile_robot=differential_drive_robot(robot_width, wheel_radius, init_pos,delta_time)
    for i in range(len(omega_l)):
        mobile_robot.velocity(omega_l[i],omega_r[i])
        
    print(mobile_robot.v_l,mobile_robot.v_r)

AttributeError: 'list' object has no attribute 'round'